In [ ]:



def projector(ket):
    ket = np.expand_dims(ket, 1)
    proj = ket.dot(ket.conjugate().T)
    return proj

sq = 1 / np.sqrt(3)
w_state = np.array([0, sq, sq, 0, sq, 0, 0, 0])
W = projector(w_state)

dev = qml.device("default.qubit", wires=3)
@qml.qnode(dev)
def circuit(params):
    qml.RZ(np.pi, wires=[0])
    qml.PauliX(wires=[2])
    qml.RY(params[0], wires=[1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[1,0])
    qml.RY(params[1], wires=[0])
    qml.CNOT(wires=[1,0])
    qml.Rot(params[2:][0],params[2:][1],params[2:][2],wires=[0])
    qml.CNOT(wires=[0,1])
    #return qml.probs(wires=range(3))
    return qml.expval(qml.Hermitian(W,wires=[0,1,2]))


sq = 1 / np.sqrt(3)
w_state = np.array([0, sq, sq, 0, sq, 0, 0, 0])
W = projector(w_state)

In [ ]:
circuit.arg_vars

In [ ]:
def loss(x):
    return 1-circuit(x)

opt = qml.GradientDescentOptimizer(stepsize=0.1)

steps = 100
params = np.random.sample(5)

for i in range(steps):
    params = opt.step(loss, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, loss(params)))
print("Optimized rotation angles: {}".format(params))

What I want now is to define the circuit s.t. i first append the gate that depends on continuous parameters and then I declare the dependence.

In [ ]:
import pickle 
with open('alphabet_w.pickle', 'rb') as alphabet:
    alphabet = pickle.load(alphabet)

def append_gate(alphabet, index):
    # let's say the parametrized gates are only rotations of 1 free param (how to )
    if "params" in list(alphabet[str(index)].keys()):
        if alphabet[str(index)]["gate"] == qml.Rot:
            params = alphabet[str(index)]["params"]
            return alphabet[str(index)]["gate"](params[0], params[1], params[2], wires=alphabet[str(index)]["wires"])
        else:
            return alphabet[str(index)]["gate"](alphabet[str(index)]["params"][0], wires=alphabet[str(index)]["wires"])
    else:
        return alphabet[str(index)]["gate"](wires=alphabet[str(index)]["wires"])


In [ ]:
def gett_params(alphabet, list_ops):
    p=np.array([])
    for index in list_ops:
        if "params" in list(alphabet[str(index)].keys()):
            p = np.append(p,alphabet[str(index)]["params"])
    return p

In [ ]:
pars = gett_params(alphabet,np.array([1,2,3,4]))
variables = {}
for i,k in enumerate(pars):
    variables[str(i)] = qml.variable.Variable(idx=i)

In [ ]:
def build_circuit():
    for op in list_ops:
        append_gate(alphabet, int(op))

In [ ]:
vv = list(variables.values())

In [ ]:

dev = qml.device("default.qubit", wires=3)
@qml.qnode(device=dev)
def circuit_obs(vv):
    build_circuit()
    return qml.expval(qml.Hermitian(W,wires=[0,1,2]))

circuit_obs(pars)

In [ ]:
#OK this not gonna work, since i have to pass the params as a list of parameters. The best way to do this is 

In [ ]:
circuit_obs.arg_vars

In [ ]:
params=[np.random.random()]

In [ ]:
dev = qml.device("default.qubit", wires=3)
@qml.qnode(dev)
def circuit(params):
    qml.RZ(np.pi, wires=[0])
    qml.PauliX(wires=[2])
    qml.RY(params[0], wires=[1])
    qml.CNOT(wires=[1,2])
    return qml.expval(qml.Hermitian(W,wires=[0,1,2]))



In [1]:
import pennylane as qml
import gym
from pennylane import numpy as np
from gym import spaces

In [20]:
alphabet_w = {"0":{"gate": qml.PauliX, "wires": [2]}, 
            "1":{"gate": qml.RZ, "wires": [0]},
            "2":{"gate": qml.RY, "wires": [1]},
            "3":{"gate": qml.CNOT, "wires": [1,2]},#, "params":[np.pi]},
            "4":{"gate": qml.CNOT, "wires": [1,0]},#, "params":[np.pi]},
            "5":{"gate": qml.RY, "wires": [0]},
            "6":{"gate":qml.Rot, "wires":[0]}, #borrowed from other optimization
            "7":{"gate": qml.CNOT, "wires": [0,1]},#, "params":[np.pi]},
           }


def projector(ket):
    ket = np.expand_dims(ket, 1)
    proj = ket.dot(ket.conjugate().T)
    return proj

sq = 1 / np.sqrt(3)
w_state = np.array([0, sq, sq, 0, sq, 0, 0, 0])
W = projector(w_state)

In [21]:
traj = np.array([3,1,0,6,6])

In [27]:
parametros=np.array([])
to_yield = []
for k in traj:
    op = alphabet_w[str(k)]
    num_params_op = op["gate"].num_params
    if num_params_op >0:
        parametros_this_gate = [qml.variable.Variable(idx=k+len(parametros)) for k in range(num_params_op)]
        parametros = np.append(parametros, parametros_this_gate)
        if len(parametros_this_gate) == 3:
            to_yield.append(op["gate"](parametros_this_gate[0],parametros_this_gate[1], parametros_this_gate[2], wires=op["wires"]))
        else:
            to_yield.append(op["gate"](parametros_this_gate[0], wires=op["wires"]))
    else:
        to_yield.append(op["gate"](wires=op["wires"]))

In [28]:
dev = qml.device("default.qubit", wires=3)
@qml.qnode(dev)
def circuit(parametros):
    for pp in to_yield:
        pp
    return qml.expval(qml.Hermitian(W,wires=[0,1,2]))

In [34]:
circuit(parametros)

0.0

In [38]:
print(circuit.draw)

<bound method BaseQNode.draw of <QNode (differentiable): device='default.qubit', func=circuit, wires=3, interface=autograd>>
